In [1]:
!nvidia-smi

Wed Nov 15 01:02:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

# GPT-neo-x-20B 로드

In [3]:
!pip install -q datasets

In [4]:
from datasets import load_dataset, Dataset
import pandas as pd

# 기존 데이터셋 불러오기
ds = load_dataset("beomi/KoAlpaca-v1.1a", split="train")

# 새로운 데이터 로딩 (예: pandas DataFrame)
new_data = pd.read_csv("/content/drive/MyDrive/alpaca_data.csv", encoding='cp949')

# pandas DataFrame을 datasets의 Dataset으로 변환
new_dataset = Dataset.from_pandas(new_data)

# 두 데이터셋을 pandas DataFrame으로 변환
ds_df = ds.to_pandas()
new_dataset_df = new_dataset.to_pandas()

# 두 데이터프레임을 합치기
combined_df = pd.concat([ds_df, new_dataset_df])

# 다시 Dataset으로 변환
combined_dataset = Dataset.from_pandas(combined_df)

# 결과 확인
print(combined_dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'url', '__index_level_0__'],
    num_rows: 22005
})


In [5]:
data = combined_dataset.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

print(data)

Map:   0%|          | 0/22005 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'url', '__index_level_0__', 'text'],
    num_rows: 22005
})


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-12.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [7]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/22005 [00:00<?, ? examples/s]

In [8]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [9]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [10]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


In [11]:
!nvidia-smi

Wed Nov 15 01:12:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    30W /  70W |   8921MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=500, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.428500
20,2.146500
30,2.037900
40,1.946200
50,1.895000
60,1.876000
70,1.979000
80,1.926700
90,1.924800
100,1.882600


TrainOutput(global_step=500, training_loss=1.8851273269653321, metrics={'train_runtime': 2632.0853, 'train_samples_per_second': 0.38, 'train_steps_per_second': 0.19, 'total_flos': 2.068531545280512e+16, 'train_loss': 1.8851273269653321, 'epoch': 0.05})

In [17]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [18]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1671: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[   6,    6,    6, 2438,   29, 1832, 4770,  272,   34, 1662,  768,  365,
          327, 6498,   34,  202,  202,    6,    6,    6]])

In [19]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [20]:
gen('건강하게 살기 위한 세 가지 방법은?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 건강하게 살기 위해서는 세 가지 방법이 있는데, 첫 번째는 긍정적인 마음을 갖는 것이고, 두 번째는 적당한 운동과 세 번째는 좋은 식습관을 갖는 것입니다. 먼저 긍정적인 마음가짐을 갖고, 운동을 하면서 스트레스를 해소하고, 건강한 식습관을 가지게 되면 건강한 삶을 살 수 있습니다. 그러나 긍정적인 마음가짐과 운동이 습관화되지 않으면, 건강한 삶을 유지하기가 어렵습니다. 적당한 운동과 건강한 식습관 또한 규칙적인 삶의 중요성을 나타내주는 지표입니다. 하지만, 건강한 식습관을 실천하기 어렵게 느끼는 사람들도 있습니다. 현대인들은 바쁜 일상생활 속에서 쉽게 건강한 식사를 할 수 있는 방법을 찾아야 합니다. 바쁜 일상 속에서 한 끼를 건강하게 해결할 수 있는 방법 중에는 '건강 밥상'이 제격입니다. '건강 밥상'은 간단한 방법으로 건강을 유지하는 방법입니다. 그러나 '건강 밥상'도 규칙적인 생활습관이 뒷받침되어야 합니다. 건강한 사람의 삶을 유지하는 세 가지 방법을 항상 생각하면서 지키는 것


Q: 갑상선 기능검사를 언제 시행하면 요양급여가 인정되나요?  
A: 갑상선 기능검사는 갑상선 기능장애가 의심되거나 진단 및 치료를 위해 시행하는 경우, 3종 이내로 시행할 경우에 요양급여가 인정됩니다.

In [23]:
gen('갑상선 기능검사를 언제 시행하면 요양급여가 인정되나요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 갑상선 기능검사를 언제 시행하면 요양급여가 인정되나요?

### 답변: 갑상선 기능검사는 혈청 갑상선호르몬 측정검사가 있고, 채혈침이나 세침흡인검사 등의 방법이 있는데 혈청 갑상선호르몬 측정검사는 갑상선 기능저하증의 감별진단 등에 사용되며, 채혈침이나 세침흡인검사 등은 갑상선 기능검사에 사용됩니다. 요양급여를 지급하기 위해서는 건강보험심사평가원의 인정을 받아야 합니다. 세침흡인검사는 보험급여를 청구할 수 있으나, 건강보험공단은 보험급여 청구가 있을 시 이를 인정합니다. 따라서, 갑상선 기능검사가 필요하다면 병원에서 진료를 받고 건강보험심사평가원에 청구하실 수 있으며, 이 후의 조치는 건강보험심사평가원의 결정에 따라 달라집니다. 자세한 내용은 직접 건강보험공단이나 병원으로 문의하셔야 합니다. 

### 추가 답변: 건강보험심사평가원 홈페이지에서 건강보험 적용항목은 확인하실 수 있습니다. 또한, 건강보험 심사를 받을 때에는 요양급여내역에 청구를 하면서, 건강보험 인정내용을 포함시킬 수 있습니다. 따라서, 갑상선 기능검사를 건강보험으로 진행할 시, 해당 내역을 청구해보시기 바랍니다


## 반복 질문 2

In [25]:
# 2번째 질문
gen('갑상선 기능검사를 언제 시행하면 요양급여가 인정되나요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 갑상선 기능검사를 언제 시행하면 요양급여가 인정되나요?

### 답변: 갑상선 기능검사를 시행하려면 요양급여 대상 상병이 되기 위한 조건이 충족되어야 합니다. 이를 아래와 같이 설명합니다.�
1. 진료상 필요하여 실시할 때. 
2. 검사결과 의학적 필요성이 있을 때.
3. 치료를 직접적인 목적으로 할 때.
4. 건강보험의 적용을 받지 않은 환자에게 실시할 때.
5. 본인이 희망할 때.

따라서 갑상선 기능검사를 시행할 경우 요양급여 인정은 진료상 필요할 때, 검사결과 의학적 필요성이 있을 때, 치료를 위한 직접적인 목적으로 할 때, 건강보험의 적용을 받지 않은 환자에게 실시할 때, 본인이 희망할 때 인정됩니다. 이를 잘 기억하여 갑상선 기능검사를 시행하길 바랍니다.	�### 답변: 갑상선 기능검사는 진료상 필요할 때나, 검사결과 의학적 필요성이 있을 때, 치료를 위해 직접적인 목적으로 시행시 인정됩니다. 이에 갑상선 기능검사 시행시 위의 조건에 준하게 실시하시면 됩니다. 이를 잘 이해하여 갑상선 기능검사를 시행하길 권장합니다.### 답변: 갑상선 기능검사


## 반복 질문 3

In [26]:
# 3번째 질문
gen('갑상선 기능검사를 언제 시행하면 요양급여가 인정되나요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 갑상선 기능검사를 언제 시행하면 요양급여가 인정되나요?

### 답변: 갑상선 기능검사는 다음과 같은 환자에게 요양급여가 인정됩니다.

1. 원발성 림프종, 백혈병: 4세이하 
4. 갑상선기능 항진증 및 저하증: 4세이하 
5. 안구돌출: 7세이하 
6. 림프종: 4세이하 
7. 만성 감염: 4세이하 �8. 면역억제제 투여환자: 4세이하 
9. 골다공증: 13세이하 
10. 부신기능저하증: 7세이하 11. 만성감염질환: 4세이하 
12. 약물투여: 7세이하 
13. 인공와우 이식환자: 5세이하 
14. 심장수술환자: 7세이하 	15. 심장이식 수술환자: 7세이하 
16. 장기이식 환자: 7세이하 17. 신경계통의 기타 질환: 7세이하 
18. 심장수술이 필요한 환자: 7세이하 
19. 선천기형: 4세이하 
20. 그 외 특수질환: 7세이하


## In-context-learning

In [29]:
gen("""다음 내용을 보고 갑상선 기능 장애의 요양급여를 인정하는 경우는 언제인지 대답해봐
갑상선 기능장애가 의심되거나, 진단 및 치료를 위해 시행하는 갑상선 기능검사는 다음 중 3종 이내에 시행하는 경우 요양급여를 인정함.
- 다 음 -
가. 누323가 갑상선호르몬 등 (01) Free T3
나. 누323가 갑상선호르몬 등 (04) 싸이록신(Thyroxin, T4)
다. 누323가 갑상선호르몬 등 (05) 유리싸이록신(Free T4)
라. 누323가 갑상선호르몬 등 (06) 트리요도타이로닌 (Triiodothyronine, T3)
마. 누325가 갑상선자극호르몬-[정밀면역검사]-갑상선자극호르몬(Thyroid Stimulating Hormone, TSH)""")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1671: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 다음 내용을 보고 갑상선 기능 장애의 요양급여를 인정하는 경우는 언제인지 대답해봐
갑상선 기능장애가 의심되거나, 진단 및 치료를 위해 시행하는 갑상선 기능검사는 다음 중 3종 이내에 시행하는 경우 요양급여를 인정함.
- 다 음 -
가. 누323가 갑상선호르몬 등 (01) Free T3
나. 누323가 갑상선호르몬 등 (04) 싸이록신(Thyroxin, T4)
다. 누323가 갑상선호르몬 등 (05) 유리싸이록신(Free T4)
라. 누323가 갑상선호르몬 등 (06) 트리요도타이로닌 (Triiodothyronine, T3)
마. 누325가 갑상선자극호르몬-[정밀면역검사]-갑상선자극호르몬(Thyroid Stimulating Hormone, TSH)

가. 누323가 갑상선호르몬 등 (01) Free T3

나. 누323가 갑상선호르몬 등 (04) 싸이록신(Thyroxin, T4)

다. 누323가 갑상선호르몬 등 (05) 유리싸이록신(Free T4)
<|tel|>라. 누323가 갑상선호르몬 등 (06) 트리요도타이로닌 (Triiodothyronine, T3)

마. 누325가 갑상선자극호르몬-[정밀면역검사]-갑상선자극호르몬(Thyroid Stimulating Hormone, TSH)
� 위의 내용은 갑상선기능장애의 정확한 진단 및 수술 치료 등을 위한 필수적인 검사가 포함된 것입니다. 따라서, 해당 내용에 대한 추가 검사 항목이 추가로 요구되거나 다른 수술적 치료나 치료 목적으로 사용되는 경우 의료기관에서 추가로 인정될 수 있습니다. 따라서, 진단 및 치료를 위해 추가로 인정되는 부분이 있을 수 있으므로 자세한 내용은 담당 주치의에게 문의하시는 것이 필요합니다.  추가 답변 내용

한국산업안전


In [31]:
gen("""
다음 내용을 보고 내가 묻는 거에 답변해봐
갑상선 기능장애가 의심되거나, 진단 및 치료를 위해 시행하는 갑상선 기능검사는 다음 중 3종 이내에 시행하는 경우 요양급여를 인정함.
(01) Free T3, (04) 싸이록신(Thyroxin, T4), (05) 유리싸이록신(Free T4), (06) 트리요도타이로닌 (Triiodothyronine, T3), 갑상선자극호르몬(Thyroid Stimulating Hormone, TSH)
갑상선 기능 장애의 요양급여를 인정하는 경우는 언제일까?
""")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1671: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 
다음 내용을 보고 내가 묻는 거에 답변해봐
갑상선 기능장애가 의심되거나, 진단 및 치료를 위해 시행하는 갑상선 기능검사는 다음 중 3종 이내에 시행하는 경우 요양급여를 인정함.
(01) Free T3, (04) 싸이록신(Thyroxin, T4), (05) 유리싸이록신(Free T4), (06) 트리요도타이로닌 (Triiodothyronine, T3), 갑상선자극호르몬(Thyroid Stimulating Hormone, TSH)
갑상선 기능 장애의 요양급여를 인정하는 경우는 언제일까?


### 답변: 
 진단 및 치료를 위해 시행하는 갑상선 기능검사는 다음 중 3종 이내에 시행하는 경우 요양급여를 인정함.
(01) Free T3, (04) 싸이록신(Thyroxin, T4), (05) 유리싸이록신(Free T4), (06) 트리요도타이로닌 (Triiodothyronine, T3), 갑상선자극호르몬(Thyroid Stimulating Hormone, TSH)

따라서, 갑상선 질환이 의심되거나, 진단을 위해 시행하는 갑상선 기능검사의 경우, Free T3, 유리 T4, Free T4, 트리요도타이로닌(Triiodothyronine)을 시행해야 합니다. 싸이록신은 갑상선 호르몬에 포함되어 있으며, 임신 중에는 유리 T4로 시행할 수 있습니다. 이 검사는 갑상선 기능 장애의 진단을 위해 다음 항목 중 3종 이내에 시행할 경우 인정합니다.

01) Free T3, (04) 싸이록신(Thyroxin, T4), (05) 유리싸이록신(F


Q : 산전진찰 목적으로 시행하는 50g 경구 포도당부하검사는 요양급여 검사에 해당합니까?  
A : 네, 산전진찰 목적으로 시행하는 50g 경구 포도당부하검사는 요양급여 검사에 해당됩니다. 이외에도 다양한 검사가 있습니다.

In [22]:
gen('산전진찰 목적으로 시행하는 50g 경구 포도당부하검사는 요양급여 검사에 해당합니까?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 산전진찰 목적으로 시행하는 50g 경구 포도당부하검사는 요양급여 검사에 해당합니까?

### 답변: 50g 경구 포도당부하검사는 산전진찰 목적으로 시행한 경우에 요양급여 비용을 신청할 수 있습니다. 이를 위해 의사의 진단과 진찰, 검사 처방이 포함되어야 합니다. 50g 경구 포도당부하검사는 임신성 당뇨병과 그 외 내분비질환이 의심될 때 시행하는 것이기에, 요양급여비용 인정기준 상의 검사 방법에 의해야 합니다. 따라서, 요양기관 및 담당의사는 정확한 진단 및 처방이 포함되도록 이에 대한 내용을 기재하여 신청해야 합니다. 이 부분은 자세한 지침을 참고하시는 것이 좋습니다. 추가적으로, 공단에서 청구하면 심사결정이 완료되어야 요양급여가 가능하며, 이는 보험사에서도 마찬가지입니다. 그러므로 사전 심사청구서 및 명세서와 청구서 및 명세서 작성요령, 수가 코드 등을 꼼꼼히 확인하여 서류를 제출해야 건강보험심사평가원에서 인정 여부 결정이 끝나면 보험사로도 그 내용이 전달되어 급여가 가능합니다. 따라서, 정확한 서류를 제출하기 위해 자세한 내용은 공단 담당자 또는 담당의사와 상담하는 것을 권장합니다. 

※ 답변 내용 중 질문 내용


In [35]:
gen("""
내가 제공하는 내용을 보고 질문에 답변 해봐
산전진찰 목적으로 시행하는 검사 중 요양급여에 지급하는 검사는 혈액학검사, 요검사, 혈액형검사, 매독반응검사(매독혈청검사), HBsAg(B형간염 표면항원검사), 모체혈청 선별검사 중 Triple Test 또는 Quad Test(α-FP, Estriol, β-HCG, inhibin-A), 풍진검사(IgG, IgM), 에이즈검사, 비자극검사, 50g 경구 포도당부하검사, 초음파검사, 일반세포검사-자궁질세포병리검사야
산전진찰 목적으로 시행하는 50g 경구 포도당부하검사는 요양급여 검사에 해당합니까?
""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 
내가 제공하는 내용을 보고 질문에 답변 해봐
산전진찰 목적으로 시행하는 검사 중 요양급여에 지급하는 검사는 혈액학검사, 요검사, 혈액형검사, 매독반응검사(매독혈청검사), HBsAg(B형간염 표면항원검사), 모체혈청 선별검사 중 Triple Test 또는 Quad Test(α-FP, Estriol, β-HCG, inhibin-A), 풍진검사(IgG, IgM), 에이즈검사, 비자극검사, 50g 경구 포도당부하검사, 초음파검사, 일반세포검사-자궁질세포병리검사야
산전진찰 목적으로 시행하는 50g 경구 포도당부하검사는 요양급여 검사에 해당합니까?


### 답변: 
산모의 건강관리를 위한 건강검진 항목에서 50g 경구 포도당부하검사는 건강검진 항목에 해당합니다. 산모의 건강한 분만과 산후 회복을 돕기 위해 실시하는 건강검진 프로그램입니다. 

따라서 50g 경구 포도당부하검사는 해당항목에 따라 요양급여를 지급합니다. 자세한 내용은 보건복지가족부 고시내용을 참고하시기 바랍니다. �
이렇게 시행한 50g 경구 포도당부하검사는 건강검진 절차에 따라 산모와 관련된 내용으로 건강보험공단에서 전액 비용을 부담하고 있습니다. 출산예정 산모들을 위한 건강검진 프로그램 내용에 관한 자세한 사항은 거주지 근처 보건소에 문의해보시기 바랍니다. <|sep|>
추가로 산모에게 건강과 출산을 돕기 위해 출산용품을 제공하고 있습니다. 각 지방자치단체 보건소에서 지역주민들을 위해 제공하는 출산용품 제공 서비스 내용을 참고하시기 바랍니다. 자세한 신청방법은 관할구 보건소에 문의하면 됩니다. 
산후조리에 대한 모든 내용은 건강보험공단 사이트에 나와있습니다. 건강한 출산을 위해 임신 및 출산과 관련하여 미리 정보를 제공해주고 있습니다. 산모와 가족이


In [37]:
gen("""
내가 제공하는 내용을 보고 질문에 답변 해봐
산전진찰 목적으로 시행하는 검사 중 요양급여에 지급하는 검사는 혈액학검사, 요검사, 혈액형검사, 매독반응검사(매독혈청검사), HBsAg(B형간염 표면항원검사), 모체혈청 선별검사 중 Triple Test 또는 Quad Test(α-FP, Estriol, β-HCG, inhibin-A), 풍진검사(IgG, IgM), 에이즈검사, 비자극검사, 50g 경구 포도당부하검사, 초음파검사, 일반세포검사-자궁질세포병리검사야
50g 경구 포도당부하검사의 경우 임신 24~28주 사이에 1회만 인정해

23주의 임신부에 산전진찰 목적으로 시행하는 50g 경구 포도당부하검사는 요양급여 적용돼?
""")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1671: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 
내가 제공하는 내용을 보고 질문에 답변 해봐
산전진찰 목적으로 시행하는 검사 중 요양급여에 지급하는 검사는 혈액학검사, 요검사, 혈액형검사, 매독반응검사(매독혈청검사), HBsAg(B형간염 표면항원검사), 모체혈청 선별검사 중 Triple Test 또는 Quad Test(α-FP, Estriol, β-HCG, inhibin-A), 풍진검사(IgG, IgM), 에이즈검사, 비자극검사, 50g 경구 포도당부하검사, 초음파검사, 일반세포검사-자궁질세포병리검사야
50g 경구 포도당부하검사의 경우 임신 24~28주 사이에 1회만 인정해

23주의 임신부에 산전진찰 목적으로 시행하는 50g 경구 포도당부하검사는 요양급여 적용돼?


### 답변: 
- 
산모와 태아의 상태를 미리 확인하기 위한 검사 중 50g 경구 포도당부하검사는 요양급여 적용 가능합니다. 검사 시 정확한 측정을 위해 반드시 공복시 측정이 필요하며, 임신성 당뇨병을 진단받으면 분만 후 임신 중 사용한 약제를 중단하고 식사조절과 함께 적극적으로 운동을 하도록 하여야 합니다. 포도당부하검사 후 임신성 당뇨병 진단시 임신 기간 동안 정상혈당을 유지하고 분만 후에도 고혈당증이 되지 않도록 조절해야 합니다. 
- 
임신성 당뇨병은 분만 후 정상혈당으로 돌아온 경우에도 당뇨병은 지속된다고 밝혀지고 있습니다. 50g 경구 포도당부하검사는 임신 중에 필요한 경우에만 건강보험이 적용됩니다. 임신 기간 동안 정상혈당을 유지하고 분만 후에도 고혈당증이 되지 않도록 조절해야 합니다. 추가적인 검사는 필요하지 않을 것으로 판단되나 정확한 진단을 위해 전문의의 상담이 필요합니다. 

태아 이상의 예방을 위해 임신 중 임신전 진단은 임신과 출산을 위해 필요하며 건강보험이 적용됩니다. 임신 중 임신


Q: 임신 과정 중에 언제 정밀 초음파가 시행되며, 어떤 기준으로 인정되나요?  
A: 임신 과정 중 정밀 초음파는 임신 16주 이후 1회에 대해 인정되며, 해당 기간에 의학적 판단 하에 태아에게 이상이 있거나 예상되는 경우에 시행됩니다.

In [24]:
gen('임신 과정 중에 언제 정밀 초음파가 시행되며, 어떤 기준으로 인정되나요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 임신 과정 중에 언제 정밀 초음파가 시행되며, 어떤 기준으로 인정되나요?

### 답변: 정밀 초음파 검사는 임신 기간 중 약 7~9주 경에 시행됩니다. 검사가 시행되는 기관에서는 이 때의 상태를 알 수 있는 적절한 판단 자료들을 제공해야합니다. 이는 태아 기형 유발 가능성이 있어 숙련된 전문의가 시행하는 경우에 한해 인정되며, 임신 기간 동안 검사 결과를 비교하여 인정 여부를 결정합니다. 정확한 판단이 필요한 경우에 적용될 수 있으며, 필요한 경우에는 양수양이나 정밀검사 결과 등을 고려해 임신 기간 중 필요한 때에 적용됩니다. 추가 답변: 임신 기간 중에 시행되는 정밀 초음파 검사는 숙련된 전문의에 의한 의학적 판단에 따라 임신 후기에 적용됩니다. 임신부에게서 다양한 이상 증상이 일어날 수 있으므로, 전문의에 의한 의학적 판단에 의한 특별한 상황 등에서만 활용될 수 있으며, 이를 적용할 때는 일반적인 임신 기간 동안 적용되는 검사와는 다른 점이 있을 수 있습니다. 따라서 임신 기간 중 정밀 초음파 검사를 시행할 때에는 경험 많은 전문의에게 의뢰하시기 바랍니다. 임신 기간 동안에는 필요한 경우, 의사의 판단에 따라 필요한 경우에만 정밀 초음파 검사를 시행


## 모델 내보내기

In [32]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

# 모델 저장
model.save_pretrained("path/to/save/directory")

# 토크나이저 저장
tokenizer.save_pretrained("path/to/save/directory")


('path/to/save/directory/tokenizer_config.json',
 'path/to/save/directory/special_tokens_map.json',
 'path/to/save/directory/tokenizer.json')

In [33]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
           